In [ ]:
import pandas as pd
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html

In [ ]:
df = pd.read_csv('output1.csv')
df = df.iloc[:, 1:]

In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    row['location'] = str(row['location']).split()[0]
    row['location'] = str(row['location']).split(',')[0]
    df.at[index, 'location'] = row['location']

In [ ]:
df

In [ ]:
df_cities = pd.read_csv('pl.csv', encoding='utf-8')
df_cities = df_cities.iloc[:, :3]
df_cities.at[0, 'city'] = 'Warszawa'

In [ ]:
df_cities

In [ ]:
def prepare_data(df, df_cities):
    location = []
    lng = []
    lat = []
    for index, row in df.iterrows():
         if df.at[index, 'location'] in df_cities['city'].values:
                location.append(df.at[index, 'location']) 
                lng.append(df_cities.at[df_cities.index[df_cities['city'] == df.at[index, 'location']].tolist()[0], 'lng'])
                lat.append(df_cities.at[df_cities.index[df_cities['city'] == df.at[index, 'location']].tolist()[0], 'lat'])
    df1 = pd.DataFrame(data = zip(location, lat, lng), columns=['location', 'lat', 'lng'])
    dictionary = df1.to_dict('records')
    return dictionary

In [ ]:
def prepare_layout(dictionary):
    geojson = dlx.dicts_to_geojson(dictionary, lat="lat", lon="lng")
    app.layout = html.Div([
        dl.Map([
            dl.TileLayer(),
            dl.GeoJSON(data=geojson,
                       id="geojson", spiderfyOnMaxZoom=True,
                       cluster=True, superClusterOptions=dict(radius=120), zoomToBoundsOnClick=True)
        ], center=(51.7769, 19.4547), zoom=7, style={'width': '90%', 'height': '100vh', 'margin': "auto", "display": "block"}),
    ])

In [ ]:
if __name__ == '__main__':
    app = Dash()
    prepare_layout(prepare_data(df, df_cities))
    app.run_server(debug=True, use_reloader=False)